# FLIGHT DELAY VISUALIZATION & MODELING

## By Ravi Surdhar, Dec 2018

This notebook uses the 2015 Flight Delays Dataset from Kaggle, as well as the Iowa State University AWOS Download Service, and attempts to generate a model to predict flight delays and cancellations based on airport weather. 

The tech stack includes Pandas, Seaborn, Matplotlib, Scikit-Learn, and TensorFlow. Topics explored include: identifying and fixing corrupted data, big data techniques and optimizations, web scraping, data visualization, and model selection and evaluation.

### OVERVIEW
#### 1. Introduction & Data Cleanup
*Fixing Corrupted Data*

*Reduction & Optimization*
#### 2. Web Scraping & Adding Weather Data
#### 3. Visualization
#### 4. Modeling

## Data Cleanup

First, let's import the usual suspects.

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
pd.set_option('display.max_columns', 35)

In [3]:
flights = pd.read_csv('flights.csv')
airport_iata_id = pd.read_csv('airport_iata_id.csv', index_col=0)
print(flights.info(memory_usage='deep'))
flights.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5819079 entries, 0 to 5819078
Data columns (total 31 columns):
YEAR                   int64
MONTH                  int64
DAY                    int64
DAY_OF_WEEK            int64
AIRLINE                object
FLIGHT_NUMBER          int64
TAIL_NUMBER            object
ORIGIN_AIRPORT         object
DESTINATION_AIRPORT    object
SCHEDULED_DEPARTURE    int64
DEPARTURE_TIME         float64
DEPARTURE_DELAY        float64
TAXI_OUT               float64
WHEELS_OFF             float64
SCHEDULED_TIME         float64
ELAPSED_TIME           float64
AIR_TIME               float64
DISTANCE               int64
WHEELS_ON              float64
TAXI_IN                float64
SCHEDULED_ARRIVAL      int64
ARRIVAL_TIME           float64
ARRIVAL_DELAY          float64
DIVERTED               int64
CANCELLED              int64
CANCELLATION_REASON    object
AIR_SYSTEM_DELAY       float64
SECURITY_DELAY         float64
AIRLINE_DELAY          float64
LATE_AIRCRAFT

,YEAR,MONTH,DAY,DAY_OF_WEEK,FLIGHT_NUMBER,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,SCHEDULED_TIME,ELAPSED_TIME,AIR_TIME,DISTANCE,WHEELS_ON,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
count,5819079.0,5.819079e+06,5.819079e+06,5.819079e+06,5.819079e+06,5.819079e+06,5.732926e+06,5.732926e+06,5.730032e+06,5.730032e+06,5.819073e+06,5.714008e+06,5.714008e+06,5.819079e+06,5.726566e+06,5.726566e+06,5.819079e+06,5.726566e+06,5.714008e+06,5.819079e+06,5.819079e+06,1.063439e+06,1.063439e+06,1.063439e+06,1.063439e+06,1.063439e+06
mean,2015.0,6.524085e+00,1.570459e+01,3.926941e+00,2.173093e+03,1.329602e+03,1.335204e+03,9.370158e+00,1.607166e+01,1.357171e+03,1.416859e+02,1.370062e+02,1.135116e+02,8.223565e+02,1.471469e+03,7.434971e+00,1.493808e+03,1.476491e+03,4.407057e+00,2.609863e-03,1.544643e-02,1.348057e+01,7.615387e-02,1.896955e+01,2.347284e+01,2.915290e+00
std,0.0,3.405137e+00,8.783425e+00,1.988845e+00,1.757064e+03,4.837518e+02,4.964233e+02,3.708094e+01,8.895574e+00,4.980094e+02,7.521058e+01,7.421107e+01,7.223082e+01,6.077843e+02,5.221879e+02,5.638548e+00,5.071647e+02,5.263197e+02,3.927130e+01,5.102012e-02,1.233201e-01,2.800368e+01,2.143460e+00,4.816164e+01,4.319702e+01,2.043334e+01
min,2015.0,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,-8.200000e+01,1.000000e+00,1.000000e+00,1.800000e+01,1.400000e+01,7.000000e+00,2.100000e+01,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,-8.700000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,2015.0,4.000000e+00,8.000000e+00,2.000000e+00,7.300000e+02,9.170000e+02,9.210000e+02,-5.000000e+00,1.100000e+01,9.350000e+02,8.500000e+01,8.200000e+01,6.000000e+01,3.730000e+02,1.054000e+03,4.000000e+00,1.110000e+03,1.059000e+03,-1.300000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,2015.0,7.000000e+00,1.600000e+01,4.000000e+00,1.690000e+03,1.325000e+03,1.330000e+03,-2.000000e+00,1.400000e+01,1.343000e+03,1.230000e+02,1.180000e+02,9.400000e+01,6.470000e+02,1.509000e+03,6.000000e+00,1.520000e+03,1.512000e+03,-5.000000e+00,0.000000e+00,0.000000e+00,2.000000e+00,0.000000e+00,2.000000e+00,3.000000e+00,0.000000e+00
75%,2015.0,9.000000e+00,2.300000e+01,6.000000e+00,3.230000e+03,1.730000e+03,1.740000e+03,7.000000e+00,1.900000e+01,1.754000e+03,1.730000e+02,1.680000e+02,1.440000e+02,1.062000e+03,1.911000e+03,9.000000e+00,1.918000e+03,1.917000e+03,8.000000e+00,0.000000e+00,0.000000e+00,1.800000e+01,0.000000e+00,1.900000e+01,2.900000e+01,0.000000e+00
max,2015.0,1.200000e+01,3.100000e+01,7.000000e+00,9.855000e+03,2.359000e+03,2.400000e+03,1.988000e+03,2.250000e+02,2.400000e+03,7.180000e+02,7.660000e+02,6.900000e+02,4.983000e+03,2.400000e+03,2.480000e+02,2.400000e+03,2.400000e+03,1.971000e+03,1.000000e+00,1.000000e+00,1.134000e+03,5.730000e+02,1.971000e+03,1.331000e+03,1.211000e+03


In [4]:
flights.head()

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,SCHEDULED_TIME,ELAPSED_TIME,AIR_TIME,DISTANCE,WHEELS_ON,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
0,2015,1,1,4,AS,98,N407AS,ANC,SEA,5,2354.0,-11.0,21.0,15.0,205.0,194.0,169.0,1448,404.0,4.0,430,408.0,-22.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
1,2015,1,1,4,AA,2336,N3KUAA,LAX,PBI,10,2.0,-8.0,12.0,14.0,280.0,279.0,263.0,2330,737.0,4.0,750,741.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
2,2015,1,1,4,US,840,N171US,SFO,CLT,20,18.0,-2.0,16.0,34.0,286.0,293.0,266.0,2296,800.0,11.0,806,811.0,5.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
3,2015,1,1,4,AA,258,N3HYAA,LAX,MIA,20,15.0,-5.0,15.0,30.0,285.0,281.0,258.0,2342,748.0,8.0,805,756.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,2015,1,1,4,AS,135,N527AS,SEA,ANC,25,24.0,-1.0,11.0,35.0,235.0,215.0,199.0,1448,254.0,5.0,320,259.0,-21.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


Wow, 2.6 GB! Let's start by whittling that down. First, we can drop columns that we don't think will provide any correlation to how delayed a flight will be.

In [5]:
flights_reduced1 = flights.drop(['FLIGHT_NUMBER', 'TAIL_NUMBER', 'TAXI_OUT', 'WHEELS_OFF', 'SCHEDULED_TIME', 'ELAPSED_TIME', 'AIR_TIME', 'DISTANCE', 'WHEELS_ON', 'TAXI_IN', 'AIR_SYSTEM_DELAY', 'SECURITY_DELAY', 'AIRLINE_DELAY', 'LATE_AIRCRAFT_DELAY', 'WEATHER_DELAY'], axis=1)
flights_reduced1.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5819079 entries, 0 to 5819078
Data columns (total 16 columns):
YEAR                   int64
MONTH                  int64
DAY                    int64
DAY_OF_WEEK            int64
AIRLINE                object
ORIGIN_AIRPORT         object
DESTINATION_AIRPORT    object
SCHEDULED_DEPARTURE    int64
DEPARTURE_TIME         float64
DEPARTURE_DELAY        float64
SCHEDULED_ARRIVAL      int64
ARRIVAL_TIME           float64
ARRIVAL_DELAY          float64
DIVERTED               int64
CANCELLED              int64
CANCELLATION_REASON    object
dtypes: float64(4), int64(8), object(4)
memory usage: 1.6 GB


That saved 1 GB right there! Good start! We'll continue to do some more memory optimizations in a moment, but for now, let's make sure our dataframe has no errors by making sure all the numeric columns make sense, and the string columns all contain what we expect. 

In [6]:
print(flights_reduced1.describe())
print(flights_reduced1['AIRLINE'].unique())
print(flights_reduced1['ORIGIN_AIRPORT'].unique())
print(flights_reduced1['DESTINATION_AIRPORT'].unique())
print(flights_reduced1['CANCELLATION_REASON'].unique())

            YEAR         MONTH           DAY   DAY_OF_WEEK  \
count  5819079.0  5.819079e+06  5.819079e+06  5.819079e+06   
mean      2015.0  6.524085e+00  1.570459e+01  3.926941e+00   
std          0.0  3.405137e+00  8.783425e+00  1.988845e+00   
min       2015.0  1.000000e+00  1.000000e+00  1.000000e+00   
25%       2015.0  4.000000e+00  8.000000e+00  2.000000e+00   
50%       2015.0  7.000000e+00  1.600000e+01  4.000000e+00   
75%       2015.0  9.000000e+00  2.300000e+01  6.000000e+00   
max       2015.0  1.200000e+01  3.100000e+01  7.000000e+00   

       SCHEDULED_DEPARTURE  DEPARTURE_TIME  DEPARTURE_DELAY  \
count         5.819079e+06    5.732926e+06     5.732926e+06   
mean          1.329602e+03    1.335204e+03     9.370158e+00   
std           4.837518e+02    4.964233e+02     3.708094e+01   
min           1.000000e+00    1.000000e+00    -8.200000e+01   
25%           9.170000e+02    9.210000e+02    -5.000000e+00   
50%           1.325000e+03    1.330000e+03    -2.000000e+00   


The numeric columns all make sense: all of the years are 2015, the months go from 1 to 12 with a mean of 7, etc. There's also the same number of entries in the YEAR column as there are in the MONTH, DAY, DAY_OF_WEEK, SCHEDULED_DEPARTURE, SCHEDULED_ARRIVAL, and CANCELLED columns, just as we'd expect (some flights wouldn't have a DEPARTURE_TIME, DEPARTURE_DELAY, etc if they're cancelled). CANCELLED makes sense too; it's a one-hot encoded, with 1 indicating that the flight was cancelled.

For the string columns, CANCELLATION_REASON is what we expect, where the different letters correspond to the reason for a cancellation, and a NaN value if the flight was not cancelled. AIRLINE is also what we'd expect, as those are two-letter identifiers that are given in the airlines.csv file. 

However, the airports are clearly corrupted. All of the entries in ORIGIN_AIRPORT and DESTINATION_AIRPORT should be three-letter IATA airport identifiers, but there are a bunch of strings of five numbers, as well as a bunch of five-digit integers. Let's fix that next.

After quite a bit of digging, it turns out that the corrupted airport entries are only for flights in Oct. So let's start by isolating those.

In [7]:
corrupted_flights = flights_reduced1[flights_reduced1['MONTH'] ==10]
corrupted_flights.head()

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON
4385712,2015,10,1,4,AA,14747,11298,5,15.0,10.0,602,543.0,-19.0,0,0,NaN
4385713,2015,10,1,4,DL,14771,13487,5,16.0,11.0,538,528.0,-10.0,0,0,NaN
4385714,2015,10,1,4,NK,12889,13487,5,2400.0,-5.0,502,448.0,-14.0,0,0,NaN
4385715,2015,10,1,4,AA,12892,13303,10,7.0,-3.0,806,813.0,7.0,0,0,NaN
4385716,2015,10,1,4,AA,14771,11057,10,8.0,-2.0,801,750.0,-11.0,0,0,NaN


Luckily, the person from the Department of Transportation who uploaded the dataset included an Airport ID.csv file, which contains a column of five-digit airport IDs and a column of names. We can merge this with a list of all airports with the IATA codes (also from the DOT) to create a dictionary of key : value pairs corresponding to ID : IATA.

In [10]:
airport_id = pd.read_csv('AIRPORT ID.csv')
airport_iata = pd.read_csv('AIRPORT IATA.csv')
airport_iata_id = pd.merge(airport_iata, airport_id, on='Description')
airport_iata_id.rename({'Code_x': 'IATA', 'Code_y': 'ID'}, axis=1, inplace=True)
airport_dictionary = airport_iata_id[['ID', 'IATA']].set_index('ID').to_dict()['IATA']

Before we can use the dictionary to replace the corrupted airports, we first need to convert all of airports that are five-digit strings to five-digit integers, because our dictionary keys are all integers. Then we can run the replacement. With tens of thousands of flights in our corrupted_flights dataframe, this replacement takes a few minutes!

In [11]:
corrupted_flights['ORIGIN_AIRPORT'] = pd.to_numeric(corrupted_flights['ORIGIN_AIRPORT'])
corrupted_flights['DESTINATION_AIRPORT'] = pd.to_numeric(corrupted_flights['DESTINATION_AIRPORT'])
fixed_flights = corrupted_flights.replace({'ORIGIN_AIRPORT' : airport_dictionary, 'DESTINATION_AIRPORT' : airport_dictionary})

Finally, we can merge our fixed_flights with our flights_reduced1 dataframe, then do a couple checks on the results.

In [12]:
merged_flights = flights_reduced1[flights_reduced1['MONTH'] < 10].append(fixed_flights).append(flights_reduced1[flights_reduced1['MONTH'] > 10])
print(merged_flights['MONTH'].unique(), '\n',               # Expect to see all 12 months
    merged_flights['ORIGIN_AIRPORT'].unique(), '\n',        # Expect to see only 3-letter IATA strings
    merged_flights['ORIGIN_AIRPORT'].nunique(), '\n',
    merged_flights['DESTINATION_AIRPORT'].unique(), '\n',   # Expect to see only 3-letter IATA strings
    merged_flights['DESTINATION_AIRPORT'].nunique())

[ 1  2  3  4  5  6  7  8  9 10 11 12] 
 ['ANC' 'LAX' 'SFO' 'SEA' 'LAS' 'DEN' 'SLC' 'PDX' 'FAI' 'MSP' 'PHX' 'SJU'
 'PBG' 'IAG' 'PSE' 'BQN' 'ORD' 'GEG' 'HNL' 'ONT' 'MCO' 'BOS' 'HIB' 'ABR'
 'MAF' 'DFW' 'MKE' 'IAH' 'BNA' 'BRO' 'VPS' 'BOI' 'BJI' 'SGF' 'PHL' 'SBN'
 'RDD' 'EUG' 'IAD' 'BUF' 'PWM' 'JFK' 'CRP' 'PIA' 'FAT' 'SMF' 'AUS' 'MCI'
 'ATL' 'JAX' 'MFR' 'IDA' 'MSN' 'DCA' 'SAT' 'CHS' 'SBA' 'SMX' 'IND' 'CLE'
 'GSP' 'BDL' 'ABI' 'RIC' 'BFL' 'OMA' 'RDM' 'FLL' 'CID' 'TPA' 'SYR' 'ROC'
 'TYR' 'LAN' 'XNA' 'GSO' 'EWR' 'PBI' 'RSW' 'OAK' 'PVD' 'RNO' 'PIT' 'ABQ'
 'MIA' 'BWI' 'LGA' 'TUL' 'LIT' 'MSY' 'OKC' 'ATW' 'PNS' 'MEM' 'TYS' 'MHT'
 'SAV' 'CLT' 'GRB' 'ABE' 'JAN' 'OAJ' 'FAR' 'ERI' 'LEX' 'CWA' 'MSO' 'TTN'
 'AMA' 'CLL' 'HOU' 'JLN' 'MLI' 'RDU' 'CVG' 'MHK' 'MOB' 'TLH' 'BHM' 'CAE'
 'TXK' 'ACY' 'DTW' 'RAP' 'TUS' 'EAU' 'DLH' 'FSD' 'INL' 'CMX' 'SPI' 'CLD'
 'COD' 'CMH' 'LRD' 'PSC' 'CPR' 'ACV' 'DAL' 'PAH' 'MRY' 'ESC' 'ISN' 'PSP'
 'MFE' 'STL' 'BTV' 'FSM' 'AEX' 'SPS' 'ACT' 'SJT' 'MTJ' 'GCC' 'OGG' 'SJC'
 'GUC' 'ORF

Looks good! Our next step will be to only include flights in our dataframe that are the in the top 1000 most travelled routes. Airports with only a few flights will most likely create outliers in our data that will throw off our modeling. 

In [13]:
flights_grouped = flights.groupby(['ORIGIN_AIRPORT', 'DESTINATION_AIRPORT'])
top_1000 = flights_grouped.size().reset_index(name='counts').sort_values(by='counts', ascending=False).head(1000)
top_airports = np.unique(np.append(top_1000['ORIGIN_AIRPORT'].unique(), top_1000['DESTINATION_AIRPORT'].unique()))
flights_reduced2 = flights_reduced1[flights_reduced1['ORIGIN_AIRPORT'].isin(top_airports) & flights_reduced1['DESTINATION_AIRPORT'].isin(top_airports)]
print(top_1000.head())
print(top_1000.tail())

      ORIGIN_AIRPORT DESTINATION_AIRPORT  counts
11867            SFO                 LAX   13744
10165            LAX                 SFO   13457
9949             JFK                 LAX   12016
10130            LAX                 JFK   12015
10053            LAS                 LAX    9715
      ORIGIN_AIRPORT DESTINATION_AIRPORT  counts
9970             JFK                 SAN    1710
9256             EWR                 LAS    1707
9085             DTW                 DCA    1705
8271             BWI                 ALB    1704
11650            SAN                 IAH    1701


This looks reasonable. Our least common route in our dataframe has fewer than 5 flights per day, and our most common has 37 flights per day, so this spans a little less than one order of magnitude. (In retrospect, using only the top 1000 routes is a little arbitrary. I could have used only routes within one order of magnitude of the most common route. Also, there are 27132 routes with less than one flight per day!) Let's take a look at our new flights_reduced2 dataframe.

In [14]:
print(flights_reduced2.describe())
print(flights_reduced2.info(memory_usage='deep'))

            YEAR         MONTH           DAY   DAY_OF_WEEK  \
count  4764554.0  4.764554e+06  4.764554e+06  4.764554e+06   
mean      2015.0  6.213215e+00  1.569010e+01  3.916054e+00   
std          0.0  3.383147e+00  8.774449e+00  1.992848e+00   
min       2015.0  1.000000e+00  1.000000e+00  1.000000e+00   
25%       2015.0  3.000000e+00  8.000000e+00  2.000000e+00   
50%       2015.0  6.000000e+00  1.600000e+01  4.000000e+00   
75%       2015.0  9.000000e+00  2.300000e+01  6.000000e+00   
max       2015.0  1.200000e+01  3.100000e+01  7.000000e+00   

       SCHEDULED_DEPARTURE  DEPARTURE_TIME  DEPARTURE_DELAY  \
count         4.764554e+06    4.693421e+06     4.693421e+06   
mean          1.329576e+03    1.335907e+03     1.001457e+01   
std           4.845708e+02    4.980484e+02     3.736194e+01   
min           1.000000e+00    1.000000e+00    -6.800000e+01   
25%           9.150000e+02    9.180000e+02    -5.000000e+00   
50%           1.325000e+03    1.330000e+03    -1.000000e+00   


We've reduced the number of rows by nearly 20% and the memory by 13%, but we can still do a lot better!


## Reduction & Optimization

One big consideration when working with large dataframes is to be efficient with your data types. Here, we can see that YEAR, MONTH, DAY, and DAY_OF_WEEK are all 8-byte int64, which is way bigger than we need. Even our one-hot CANCELLED column is an int64! Let's start by downcasting all of the numeric rows (note that the times are all integer values,  but they're required to be floats as ints cannot store NaN values.

In [15]:
flights_reduced2_int = flights_reduced2.select_dtypes(include=['int64'])
converted_int = flights_reduced2_int.apply(pd.to_numeric,downcast='unsigned')

flights_reduced2_float = flights_reduced2.select_dtypes(include=['float64'])
converted_float = flights_reduced2_float.apply(pd.to_numeric,downcast='float')

optimized_flights_reduced2 = flights_reduced2.copy()

optimized_flights_reduced2[converted_int.columns] = converted_int
optimized_flights_reduced2[converted_float.columns] = converted_float

print(flights_reduced2.info(memory_usage='deep', verbose=False))
print(optimized_flights_reduced2.info(memory_usage='deep', verbose=False))

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4764554 entries, 0 to 5819077
Columns: 16 entries, YEAR to CANCELLATION_REASON
dtypes: float64(4), int64(8), object(4)
memory usage: 1.4 GB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 4764554 entries, 0 to 5819077
Columns: 16 entries, YEAR to CANCELLATION_REASON
dtypes: float32(4), object(4), uint16(3), uint8(5)
memory usage: 1.1 GB
None


That's another 22% reduction! Not bad, but we can do better. Way better. 

Our four Object (string) columns have very few unique entries in each column. That's a perfect canditate to converting to a Catagory type, which Pandas treats as an int under the hood (see this great article for more info: https://www.dataquest.io/blog/pandas-big-data/)

In [16]:
flights_reduced2_obj = flights_reduced2.select_dtypes(include=['object']).copy()
print(flights_reduced2_obj.describe())

flights_cat = flights_reduced2_obj.astype('category')
print(flights_reduced2_obj.info(memory_usage='deep', verbose=False))
print(flights_cat.info(memory_usage='deep', verbose=False))

        AIRLINE ORIGIN_AIRPORT DESTINATION_AIRPORT CANCELLATION_REASON
count   4764554        4764554             4764554               74022
unique       14            134                 134                   4
top          WN            ATL                 ATL                   B
freq    1118581         309319              309398               39850
<class 'pandas.core.frame.DataFrame'>
Int64Index: 4764554 entries, 0 to 5819077
Columns: 4 entries, AIRLINE to CANCELLATION_REASON
dtypes: object(4)
memory usage: 997.2 MB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 4764554 entries, 0 to 5819077
Columns: 4 entries, AIRLINE to CANCELLATION_REASON
dtypes: category(4)
memory usage: 63.6 MB
None


997 MB down to 64 MB! That's a 94% reduction! But we still have one more trick up our sleeve!

We can combine the first 5 columns of our dataframe (YEAR, MONTH, DAY, DAY_OF_WEEK, and SCHEDULED_DEPARTURE) into a single Pandas Timestamp object (Panda's implementation of Python's DateTime type). We can do the same for the SCHEDULED_ARRIVAL. Then we can drop the DEPARTURE_TIME and ARRIVAL_TIME columns, since those columns are equal to SCHEDULED_DEPARTURE + DEPARTURE_DELAY and SCHEDULED_ARRIVAL + ARRIVAL_DELAY, respectively.

In [17]:
# Make a df containing the Year, Month, Day, Hour, and Minute of the Scheduled Departure time, then convert that to a single column of Timestamps
df = pd.concat([flights_reduced2[['YEAR', 'MONTH', 'DAY']], np.floor(flights_reduced2['SCHEDULED_DEPARTURE']/100), flights_reduced2['SCHEDULED_DEPARTURE']%100], axis=1)
df.columns = ['Year', 'Month', 'Day', 'Hour', 'Minute']
flights_reduced2['SCHEDULED_DEPARTURE_DATETIME'] = pd.to_datetime(df)

# Do the same for the Scheduled Arrival time
df = pd.concat([flights_reduced2[['YEAR', 'MONTH', 'DAY']], np.floor(flights_reduced2['SCHEDULED_ARRIVAL']/100), flights_reduced2['SCHEDULED_ARRIVAL']%100], axis=1)
df.columns = ['Year', 'Month', 'Day', 'Hour', 'Minute']
flights_reduced2['SCHEDULED_ARRIVAL_DATETIME'] = pd.to_datetime(df)

flights_reduced2.drop(['YEAR', 'MONTH', 'DAY', 'DAY_OF_WEEK', 'SCHEDULED_DEPARTURE', 'DEPARTURE_TIME', 'SCHEDULED_ARRIVAL', 'ARRIVAL_TIME'], axis=1, inplace=True)
flights_reduced2.rename({'SCHEDULED_DEPARTURE_DATETIME' : 'SCHEDULED_DEPARTURE', 'SCHEDULED_ARRIVAL_DATETIME' : 'SCHEDULED_ARRIVAL'}, axis=1, inplace=True)
flights_reduced2.head()

,AIRLINE,ORIGIN_AIRPORT,DESTINATION_AIRPORT,DEPARTURE_DELAY,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,SCHEDULED_DEPARTURE,SCHEDULED_ARRIVAL
0,AS,ANC,SEA,-11.0,-22.0,0,0,NaN,2015-01-01 00:05:00,2015-01-01 04:30:00
1,AA,LAX,PBI,-8.0,-9.0,0,0,NaN,2015-01-01 00:10:00,2015-01-01 07:50:00
2,US,SFO,CLT,-2.0,5.0,0,0,NaN,2015-01-01 00:20:00,2015-01-01 08:06:00
3,AA,LAX,MIA,-5.0,-9.0,0,0,NaN,2015-01-01 00:20:00,2015-01-01 08:05:00
4,AS,SEA,ANC,-1.0,-21.0,0,0,NaN,2015-01-01 00:25:00,2015-01-01 03:20:00


At this point, it would be convenient to write a new .csv file with only our flights_reduced2 columns. We can also store all of the optimized data types into a dictionary so that if we have to shutdown the python kernen and start it again, we can read flights.csv directly into the optimized data types. 

In [18]:
flights_reduced2.to_csv('flights_reduced.csv')
optimized_flights_reduced2.drop(['YEAR', 'MONTH', 'DAY', 'DAY_OF_WEEK', 'SCHEDULED_DEPARTURE', 'DEPARTURE_TIME', 'SCHEDULED_ARRIVAL', 'ARRIVAL_TIME'], axis=1, inplace=True)
dtypes = optimized_flights_reduced2.dtypes
dtypes_col = dtypes.index
dtypes_type = [i.name for i in dtypes.values]
column_types = dict(zip(dtypes_col, dtypes_type))
print(dtypes_col , '\n', column_types)

Index(['AIRLINE', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT', 'DEPARTURE_DELAY',
       'ARRIVAL_DELAY', 'DIVERTED', 'CANCELLED', 'CANCELLATION_REASON'],
      dtype='object') 
 {'AIRLINE': 'object', 'ORIGIN_AIRPORT': 'object', 'DESTINATION_AIRPORT': 'object', 'DEPARTURE_DELAY': 'float32', 'ARRIVAL_DELAY': 'float32', 'DIVERTED': 'uint8', 'CANCELLED': 'uint8', 'CANCELLATION_REASON': 'object'}


Let's read that .csv back in as a new flights dataframe.

In [20]:
flights = pd.read_csv('flights_reduced.csv', 
                      index_col=0, 
                      dtype = {'AIRLINE': 'category', 
                                'ORIGIN_AIRPORT': 'category', 
                                'DESTINATION_AIRPORT': 'category', 
                                'DEPARTURE_DELAY': 'float32', 
                                'ARRIVAL_DELAY': 'float32', 
                                'DIVERTED': 'uint8', 
                                'CANCELLED': 'uint8', 
                                'CANCELLATION_REASON': 'category'}, 
                      parse_dates = ['SCHEDULED_DEPARTURE', 'SCHEDULED_ARRIVAL']).reset_index(drop=True)

In [21]:
flights.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4764554 entries, 0 to 4764553
Data columns (total 10 columns):
AIRLINE                category
ORIGIN_AIRPORT         category
DESTINATION_AIRPORT    category
DEPARTURE_DELAY        float32
ARRIVAL_DELAY          float32
DIVERTED               uint8
CANCELLED              uint8
CANCELLATION_REASON    category
SCHEDULED_DEPARTURE    datetime64[ns]
SCHEDULED_ARRIVAL      datetime64[ns]
dtypes: category(4), datetime64[ns](2), float32(2), uint8(2)
memory usage: 145.4 MB


181.8 MB down from our original 2.6 GB! Our optimized flights dataframe uses 7% of the memory that our original one did!

In the next notebook, we'll add weather data for each flight into our dataframe.